API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [13]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [14]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH


In [4]:
import oracledb
import sqlite3
import os
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

In [16]:
####################################
# 1. Oracle 데이터베이스 연결 및 조회(선택)
####################################

# SQL 파일 읽기
sql_path = '../.db/oracle/product.sql'
with open(sql_path, 'r', encoding='utf-8') as f:
    sql = f.read().strip()

# Oracle 데이터베이스 연결 정보
connection = oracledb.connect(
    user=os.getenv("ORACLE_USER"),
    password=os.getenv("ORACLE_PASSWORD"),
    host=os.getenv("ORACLE_HOST"),
    port=os.getenv("ORACLE_PORT"),
    sid=os.getenv("ORACLE_SID")
    )

cursor = connection.cursor()
cursor.execute(sql)
rows = cursor.fetchall()

# 연결 종료
cursor.close()
connection.close()

In [3]:
####################################
# 1. sqlite 데이터베이스 연결 및 조회(선택)
####################################
sql = "SELECT * FROM PRODUCT"
# SQLite 데이터베이스 연결
conn = sqlite3.connect('../.db/sqlite/my_database.db')
cursor = conn.cursor()

# SQL 쿼리 실행
cursor.execute(sql)
rows = cursor.fetchall()

# 연결 종료
cursor.close()
conn.close()

NameError: name 'sqlite3' is not defined

In [1]:
# 2. 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합
# 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합

# 컬럼 인덱스 정의 (실제 CSV 파일의 컬럼 순서에 맞게 조정해야 합니다)
# 이 부분은 실제 데이터의 컬럼 순서와 일치해야 합니다.
COLUMN_INDICES = {
    "GOODS_NO": 0,
    "GOODS_NM": 1,
    "GOODS_STAT_SCT_NM": 2,
    "BRND_NM": 3,
    "ARTC_INFO": 4,
    "OPT_DISP_NM": 5,
    "OPT_VAL_DESC": 6,
    "CATEGORY_NMS": 7,
    "SALE_PRC": 8,
    "DSCNT_SALE_PRC": 9,
    "CARD_DC_RATE": 10,
    "CARD_DC_NAME": 11,
    "CARD_DC_NAME_LIST": 12,
    "MAX_BENEFIT_PRICE": 13
}

def generate_search_text_from_tuple(product_data_tuple):
    """
    주어진 상품 정보 튜플에서 검색에 최적화된 통합 텍스트 문자열을 생성합니다.
    머신러닝 기반의 의미론적 검색에 유리하도록 문맥을 구성합니다.

    Args:
        product_data_tuple (tuple): 상품 정보를 담고 있는 튜플.
                                  COLUMNS_INDICES에 정의된 순서와 일치해야 합니다.

    Returns:
        str: 검색 인덱싱을 위한 상품 정보를 결합한 문자열.
    """

    search_components = []

    # 튜플에서 데이터 추출 (COLUMN_INDICES를 활용하여 가독성을 높입니다)
    goods_nm = product_data_tuple[COLUMN_INDICES["GOODS_NM"]].strip()
    goods_stat_sct_nm = product_data_tuple[COLUMN_INDICES["GOODS_STAT_SCT_NM"]].strip()
    brnd_nm = product_data_tuple[COLUMN_INDICES["BRND_NM"]].strip()
    artc_info = product_data_tuple[COLUMN_INDICES["ARTC_INFO"]].strip()
    opt_disp_nm = product_data_tuple[COLUMN_INDICES["OPT_DISP_NM"]].strip()
    opt_val_desc = product_data_tuple[COLUMN_INDICES["OPT_VAL_DESC"]].strip()
    category_nms = product_data_tuple[COLUMN_INDICES["CATEGORY_NMS"]].strip()
    
    # 숫자 값은 형 변환 시 에러 방지를 위해 get 대신 직접 접근 후 try-except 처리
    sale_prc_str = product_data_tuple[COLUMN_INDICES["SALE_PRC"]]
    dscnt_sale_prc_str = product_data_tuple[COLUMN_INDICES["DSCNT_SALE_PRC"]]
    card_dc_rate_str = product_data_tuple[COLUMN_INDICES["CARD_DC_RATE"]]
    max_benefit_price_str = product_data_tuple[COLUMN_INDICES["MAX_BENEFIT_PRICE"]]
    
    sale_prc = int(sale_prc_str) if sale_prc_str else None
    dscnt_sale_prc = int(dscnt_sale_prc_str) if dscnt_sale_prc_str else None
    card_dc_rate = int(card_dc_rate_str) if card_dc_rate_str else None
    max_benefit_price = int(max_benefit_price_str) if max_benefit_price_str else None

    card_dc_name = product_data_tuple[COLUMN_INDICES["CARD_DC_NAME"]].strip()


    # 1. 브랜드명 + 품목 정보 (카테고리/ARTC_INFO) + 상품명 조합
    # 상품의 핵심적인 정체성을 명확히 합니다.
    if brnd_nm and goods_nm:
        # ARTC_INFO에서 품목의 구체적인 타입을 추론하여 문맥에 더합니다.
        item_type = ''
        if artc_info:
            # '>' 기호로 구분된 품목 정보 중 가장 마지막 부분을 가져와 특정 품목 타입을 얻습니다.
            item_type_parts = [part.strip() for part in artc_info.split('>') if part.strip()]
            if item_type_parts:
                item_type = item_type_parts[-1]
                # 'TV 기타'와 같은 모호한 표현을 'TV 거치대' 등으로 명확히 합니다.
                if item_type == 'TV 기타':
                    item_type = 'TV 거치대'
                elif item_type.endswith('TV'): # 'UHD TV' 등 TV 종류를 유지합니다.
                    item_type = item_type
                elif item_type == '기타': # A/V 기타 > 기타와 같은 경우를 처리합니다.
                    item_type = '기타 가전제품' # 좀 더 명확한 표현을 사용합니다.
                elif '스탠드' in item_type or '거치대' in item_type: # 예: 이젤형 TV스탠드, 모니터암
                    item_type = item_type
                else: # ARTC_INFO가 너무 일반적인 경우, 품목명이나 카테고리에서 가져올 수도 있습니다.
                    if category_nms:
                        main_category_parts = [part.strip() for part in category_nms.split('>')]
                        if main_category_parts and main_category_parts[0] != 'TV·영상가전': # 이미 TV인 경우 제외
                            item_type = main_category_parts[0]


        # 품목 타입이 있고, 상품명에 이미 품목 타입이 포함되어 있지 않다면 추가합니다.
        if item_type and item_type not in goods_nm:
            search_components.append(f"{brnd_nm}의 {item_type}, {goods_nm}.")
        else: # 품목 타입이 없거나 상품명에 이미 포함되어 있다면 브랜드명과 상품명만 사용합니다.
            search_components.append(f"{brnd_nm}의 {goods_nm}.")
    elif goods_nm: # 브랜드명이 없는 경우 상품명만 사용합니다.
        search_components.append(f"{goods_nm}.")


    # 2. 제품의 주요 특성 (OPT_DISP_NM 및 OPT_VAL_DESC) 조합
    if opt_disp_nm and opt_val_desc:
        display_names = [name.strip() for name in opt_disp_nm.split(',') if name.strip()]
        values = [val.strip() for val in opt_val_desc.split(',') if val.strip()]

        # 각 특성 항목과 값을 '항목: 값' 형태로 결합합니다.
        characteristics = []
        for i in range(min(len(display_names), len(values))):
            # 비어있는 항목이나 '스마트기능'처럼 반복되는 용어는 건너뜁니다.
            # '스마트기능'이 반복되는 경우, 첫 번째 '스마트기능' 뒤에 모든 스마트기능 값을 나열합니다.
            if display_names[i] == '스마트기능':
                if i > 0 and display_names[i-1] == '스마트기능':
                    continue # 이미 처리되었으므로 건너뜁니다.
                
                # 모든 스마트기능 값을 모아서 처리
                all_smart_features = []
                for j in range(i, len(display_names)):
                    if display_names[j] == '스마트기능' and j < len(values):
                        all_smart_features.append(values[j])
                    else:
                        break # 스마트기능이 끝나면 반복 중단

                if all_smart_features:
                    characteristics.append(f"스마트기능: {', '.join(all_smart_features)}")
            else: # 일반적인 특성
                if display_names[i] and values[i]:
                    characteristics.append(f"{display_names[i]}: {values[i]}")

        if characteristics:
            search_components.append("주요 특징: " + ", ".join(characteristics) + ".")

    # 3. 카테고리명 추가
    if category_nms:
        # '|' 기호를 ', '로, '>' 기호를 ' > '로 바꾸어 카테고리 계층 및 다중 카테고리를 표현합니다.
        formatted_categories = category_nms.replace('|', ', ').replace('>', ' > ')
        search_components.append(f"카테고리: {formatted_categories}.")

    # 4. 상품 상태 추가
    if goods_stat_sct_nm and goods_stat_sct_nm != '정상상품': # '정상상품'이 아닌 경우에만 추가합니다.
        search_components.append(f"상품 상태: {goods_stat_sct_nm}.")

    # 5. 가격 및 할인 정보 추가
    price_info_components = []
    if sale_prc is not None:
        price_info_components.append(f"판매가 {sale_prc:,}원") # 천단위 콤마 추가
    if dscnt_sale_prc is not None and sale_prc is not None and dscnt_sale_prc < sale_prc:
        price_info_components.append(f"할인가 {dscnt_sale_prc:,}원") # 천단위 콤마 추가
    if card_dc_rate is not None and card_dc_rate > 0 and card_dc_name:
        if max_benefit_price is not None:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인되어 최대 {max_benefit_price:,}원") # 천단위 콤마 추가
        else:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인")

    if price_info_components:
        search_components.append("가격 정보: " + ", ".join(price_info_components) + ".")

    # 모든 구성 요소를 하나의 문자열로 결합하고 앞뒤 공백을 제거합니다.
    return " ".join(search_components).strip()

In [2]:
# documents 리스트 생성
documents = []
metadatas = []
ids = []

"""
- 예시: 삼성전자의 UHD TV, 125.7cm 4K UHD 비즈니스 TV LH50BEAHLGFXKR (설치유형 선택가능). 카테고리: TV·영상가전 > TV. 가격 정보: 판매가 559,000원.
"""
for row in rows:
    GOODS_NO = row
    metadatas.append({"GOODS_NO": GOODS_NO})
    search_text = generate_search_text_from_tuple(row)
    documents.append(search_text)
    ids.append(str(GOODS_NO))

NameError: name 'rows' is not defined

In [ ]:
# 단계 2: 문서 분할(Split Documents)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# split_documents = text_splitter.split_documents(docs)
# print(f"분할된 청크의수: {len(split_documents)}")

In [9]:
# 단계 3: 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

In [10]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.from_texts(
    texts=documents,
    embedding=embeddings,
    metadatas=metadatas,
    ids=ids
)

# 로컬에 저장
vectorstore.save_local(persist_directory)

print(f"총 {len(documents)}개의 상품 정보가 FAISS에 저장되었습니다.")

IndexError: list index out of range

In [7]:

# 4. Vectorstore 검색
vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "삼성"  # 예시 쿼리: 원하는 검색어로 변경하세요
filter = {"GOODS_NO": '0002577807'}

# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    k=5,  # 반환할 결과 수
    filter = filter # 메타데이터 필터 적용
)

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 5개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}]")
        print(f"상품정보: {doc.page_content}")
        print(f"상품번호: {doc.metadata['GOODS_NO']}")
        print(f"유사도 점수: {score}")  # 낮을수록 더 유사함
        print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")


NameError: name 'persist_directory' is not defined

In [5]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(
    search_kwargs={
        "filter": {"GOODS_NO": "0031055945"},  # 원하는 메타데이터 필터 조건
        "k": 5  # 반환할 결과 수
    }
)

NameError: name 'vectorstore' is not defined

In [37]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("제습기")

[Document(id='0002577807', metadata={'GOODS_NO': '0002577807'}, page_content='이파람의 여름제품(제습기), 산업용 제습기 PDM330C 외 2종')]

In [21]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are a product search assistant for an electronics e-commerce platform. 
Use the following pieces of retrieved context to answer the question. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [22]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [23]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성 냉장고 그거 이름이 뭐더라?"
response = chain.invoke(question)
print(response)

삼성 일반냉장고[615L] RT62A7049S9, 삼성 일반냉장고[298L] RT31CB5624C3, 일반냉장고 150L 이하 삼성전자 6년이하 A 등이 있습니다.
